In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GroupKFold,KFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error,mean_absolute_error

import math
from scipy import stats

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from keras.models import Model

From https://www.kaggle.com/code/ambrosm/tpsmar22-generalizing-the-special-values
My appologies for other contributors I missed.

In [ ]:
train_origin = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv',index_col='row_id')
test_origin = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv',index_col='row_id')

In [ ]:
train = train_origin.copy()
test = test_origin.copy()

## DATA PREPROCESSING

In [ ]:
train['time'] = pd.to_datetime(train.time)                                              
train['offical_holiday'] = train.time.dt.date.astype(str).str.contains('1991-05-27|1991-07-04|1991-09-02').astype('int')
train=train[train['offical_holiday']==0]
train=train.drop('offical_holiday',axis=1)
train=train[(train.time.dt.weekday< 4) & (train.time.dt.month > 4)]   

def preprocess(df):
    df['time'] = pd.to_datetime(df['time'])
    df['month']= df.time.dt.month
    df['day']= df.time.dt.dayofyear
    df['am'] = (df.time.dt.hour < 12) & (df.time.dt.hour >6)
    df['wkday'] = df.time.dt.weekday
    df['time']= (df.time.dt.hour-12)*4+df.time.dt.minute/20
    df['xydirday']= df.x.astype(str)+df.y.astype(str)+df.direction+df.day.astype(str)
    df['xydir'] = df.x.astype(str)+df.y.astype(str)+df.direction
    df['all']= df['xydir']+df.time.astype(str)
    df['am'] = df['am'].map({False:0,True:1})

    mapper_med = train[['all','congestion']].groupby(['all']).median().to_dict()['congestion']
    df['med']= df['all'].map(mapper_med)
    
preprocess(train)
preprocess(test)

train= train[train.time >=0]

train = pd.concat([train,pd.get_dummies(train['y'],prefix ='y'),pd.get_dummies(train['x'],prefix ='x'),pd.get_dummies(train['direction'],prefix ='dir')],axis=1).reset_index(drop=True)
test = pd.concat([test,pd.get_dummies(test['y'],prefix ='y'),pd.get_dummies(test['x'],prefix ='x'),pd.get_dummies(test['direction'],prefix ='dir')],axis = 1).reset_index(drop=True)
target = train['congestion']

train.drop(['direction','xydirday','xydir','all','congestion','x','y','day'],axis=1,inplace = True)
test.drop(['direction','xydirday','xydir','all','x','y','day'],axis=1,inplace = True)


print(train.shape,test.shape,target.shape)

train

In [ ]:
sc = StandardScaler()
train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)
train_sc

In [ ]:
LOSS = tf.keras.losses.MeanAbsoluteError(
    name='mean_absolute_error'
)

METRIC = tf.keras.metrics.MeanAbsoluteError(
    name='mean_absolute_error', dtype=None
)

es = tf.keras.callbacks.EarlyStopping(
    monitor= 'val_loss', min_delta=1e-12, patience=10, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=2, verbose=0,
    mode='auto')

In [ ]:
def NN_model():

    inp = layers.Input(shape = (20))
       
    x = layers.Dense(
            units = 100, 
            activation ='relu',
            kernel_initializer ='he_uniform')(inp)
        
    reg_outputs = layers.Dense(
            units = 1,
        activation = 'linear',
        kernel_initializer ='he_uniform',name = 'last')(x)

    model = Model(inp,reg_outputs)

    return model

In [ ]:
pred_folds_soft_vote = []
pred_train = np.zeros(shape=(train.shape[0]))

GROUPS = train['wkday']
gkf = GroupKFold(n_splits=4)
fold =0
for train_index, val_index in gkf.split(train_sc, target,groups = GROUPS) :
    fold +=1
    x_tr = train_sc[train_index]
    y_tr = target[train_index]
    x_val = train_sc[val_index]
    y_val= target[val_index]
    
    model = NN_model()
    model.compile(loss=LOSS,
                  metrics = METRIC,
                  optimizer = keras.optimizers.Adam(learning_rate=0.001))

    model.fit(x_tr,y_tr,
              batch_size = 512, 
              epochs = 100,
              validation_data=(x_val,y_val),
              callbacks=[es, plateau],
              verbose =0)
    
    pred_val = np.round(model.predict(x_val))
    score_fold = mean_absolute_error(y_val,pred_val)
    print('SCORE FOLD {} = {}'.format(fold,score_fold))
    pred_train[val_index]= pred_val.squeeze()
    
    pred = np.round(model.predict(test_sc))
    pred_folds_soft_vote.append(pred)
    
pred_folds_soft_vote = np.round(np.mean(pred_folds_soft_vote,axis=0))

score_total = mean_absolute_error(target,pred_train)
print('SCORE OOF',score_total)

# POSTPROCESS NEURAL NETWORK :

In [ ]:
train = train_origin.copy()
test = test_origin.copy()

train['time'] = pd.to_datetime(train.time)                                               
train['offical_holiday'] = train.time.dt.date.astype(str).str.contains('1991-05-27|1991-07-04|1991-09-02').astype('int')
train=train[train['offical_holiday']==0]
train=train.drop('offical_holiday',axis=1)
train=train[(train.time.dt.weekday< 4) & (train.time.dt.month > 4)]   

preprocess(train)
preprocess(test)

sep = train[(train.day >= 246) & (train.time >= 0)]
lower = sep.groupby(['time', 'x', 'y', 'direction']).congestion.quantile(0.15).values
upper = sep.groupby(['time', 'x', 'y', 'direction']).congestion.quantile(0.7).values

test['pred'] = pred_folds_soft_vote
test.pred = test.pred.clip(lower, upper)

for xydir in set(test.xydir):
    xydir_counts = train.loc[train.xydir ==xydir,'congestion'].value_counts()
    l = xydir_counts[(xydir_counts > 200)] # experimental
    if len(l) > 2: # experimental
        l = list(l.index)
        test.loc[test.xydir ==xydir,'pred']= test.loc[test.xydir ==xydir,'pred'].map(lambda y: min(l, key=lambda x:abs(x-y)))

test['pred']

## SUBMISSION NN ALONE

In [ ]:
sub = pd.DataFrame(list(zip(test.index,test.pred.values)),columns = ['row_id', 'congestion'])
sub.to_csv('sub_NN.csv', index = False) # SCORE MAE (Public LB) = 4.859
sub.head(5)

## SUBMISSION WITH BLENDING

from kotrying
https://www.kaggle.com/code/kotrying/tps-2022-03-lgbm

Blend from :

@bernhardklinger
https://www.kaggle.com/code/bernhardklinger/march-tps-lgbm

@martynovandrey
https://www.kaggle.com/code/martynovandrey/tps-mar-22-fe-model-selection/notebook



In [ ]:
sub_blend = pd.read_csv('../input/TPS-2022-03/submission_emsemble.csv')
sub_blend['congestion'] = np.round(0.9 * sub_blend['congestion'] + 0.1 * sub['congestion'])
sub_blend.to_csv('sub_blend.csv',index = False)
sub_blend